### 0. Importing Libraries

In [ ]:
import torch
import os
from safetensors.torch import load_file, save_file
from dataclasses import dataclass

`Warning`: Not all loras can be compressed using this technique. (atleast the low step loras in my experimentations).

For low step loras in fp32 format, it can be converted to fp16 format. Check the last section for it.

### 1. Download Models

#### 1.1 Text Guide

`Note`: Civitai api format for downloading models keeps on chainging. So this might look different in future.

Current Format: `https://civitai.com/api/download/models//{VersionID}?token={api_token}`

Also, any sdxl finetune will give results (choice is not important).

How to find *VersionID* and *API Key*:<br><br>
1. **VersionID**: *Model_Page (on CivitAI) -> Details (Right Side) -> AIR (Last colummn) -> VersionID (Last number)*. <br><br>
2.**API Keys**: *Civitai_account (Top Right Corner, Profile Picture) -> Account Settings (Bottom) -> API Keys (Scroll to bottom) -> API Keys Add API Key (Copy it and save it somewhere for future use, you can only access it once)*

#### 1.2 Version Id and API Token

In [ ]:
chkpt_VersionId = None #@param {type: 'integer'}
lora_VersionId = None #@param {type: 'integer'}
civitai_api_token = "" #@param {type: 'string'}

chkpt_download_path = f"https://civitai.com/api/download/models/{chkpt_VersionId}?token={civitai_api_token}"
lora_download_path = f"https://civitai.com/api/download/models/{lora_VersionId}?token={civitai_api_token}"

!wget "{chkpt_download_path}" --content-disposition -P "/content"
!wget "{lora_download_path}" --content-disposition -P "/content"

### 2. Setup Environment

`Main Github Repository`: https://github.com/elias-gaeros/resize_lora

Clone the repository and install necessary libraries.

In [ ]:
!git clone https://github.com/elias-gaeros/resize_lora.git
%cd resize_lora

### 3. Config

#### 3.1 Short Parameter Guide

`Key Arguments`:

- **model**: Path to the input LoRA file.
- **output**: Path to save the resized LoRA file.
- **new_rank**: The target rank (integer). Set to 0 for dynamic rank selection.
- **device**: cpu or cuda (for GPU acceleration, much faster for SVD).
- **mode**: svd or extract.
- **dynamic_method, dynamic_param**: Used when new_rank=0.
- **verbose**: Print more details during processing.

`Note`: Refer to orignal repository for more indepth explaination of parameters.

`Warning`: Put the exact name of downloaded models.
Tips:
- On the left hand toolbar -> Last icon Files -> Copy name with extension.
- Easier way is to hover over lora name -> click on three dots -> Copy path


Sometimes the downloaded folders have weird names, thats why it is preferable to manually input the names.

#### 3.2 Main Config

In [ ]:
chkpt_path = "/content/cyberrealisticXL_v53.safetensors" #@param {type: 'string'}
input_lora_path = "/content/Breckie_Hill-000007.safetensors" #@param {type: 'string'}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def check_paths(path):
    if not os.path.exists(path):
        print(f"⚠️ WARNING: Input file '{path}' does not exist.")
        print("Please upload your Chkpt/LoRA file and make sure the path is correct.")
    else:
        print(f"✅ Input Chkpt/LoRA found: {path}")

check_paths(chkpt_path)
check_paths(input_lora_path)

#### 3.3 Resizing

In [ ]:
command = f"python resize_lora.py \
            {chkpt_path} \
            {input_lora_path} \
            -o '' \
            -v -r spn_lora=1,thr=-0.7:spn_ckpt=1,thr=-1.2:subspace=0.5,spn_ckpt=0.5,size=32"



print(f"Executing command:\n{command}\n")
print(f"Using device: {device}")
!{command}

print("\nResizing process finished.")

`Note`: Dowload File from files in left hand toolbar. There might be multiple versions, download all and test them all in your ui.

### 4. Additional: FP32 -> FP16 Conversions

Some loras provide fp32 versions only. This can be used to convert to fp16.

In [ ]:
def fp32_fp16(input_lora_path, output_lora_path):
    fp16_state_dict = {}
    conversion_count = 0
    skipped_count = 0
    original_total_params = 0
    original_float_params = 0
    state_dict = load_file(input_lora_path, device="cpu")
    for key, tensor in state_dict.items():
        original_total_params += tensor.numel()
        if tensor.dtype == torch.float32 or tensor.dtype == torch.bfloat16:
            fp16_state_dict[key] = tensor.to(dtype=torch.float16)
            conversion_count += 1
            original_float_params += tensor.numel()
        else:
            # Keep non-float tensors (like int, bool, etc.) as they are
            fp16_state_dict[key] = tensor
            skipped_count += 1

    save_file(fp16_state_dict, output_lora_path)

fp32_fp16(f'{input_lora_path}', '/content/compressed_fp16_lora.safetensors')